# Introduction to ODSE datasets in Python

Tutorial explaing how to access several raster layers publicly available and provided by [Open Data Science Europe](https://opendatascience.eu/) through the follow plataforms:
* [ODSE Viewer](http://maps.opendatascience.eu)
* [ODSE STAC](https://stac.opendatascience.eu)

## Python environment & libraries

In this tutorial, you will use [leafmap](https://leafmap.org/) to display COG directly in Colab. This package provides geospatial analysis and interactive mapping in a Jupyter environment. Don't forget to restart the **restart runtime** at end of the installation:

In [ ]:
!pip install rasterio
!pip install leafmap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 6.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 158 kB 6.6 MB/s 
     |████████████████████████████████| 8.8 MB 19.5 MB/s 
     |████████████████████████████████| 3.0 MB 42.9 MB/s 
     |████████████████████████████████| 3.3 MB 51.1 MB/s 
     |████████████████████████████████| 95 kB 4.4 MB/s 
     |████████████████████████████████| 3.8 MB 41.4 MB/s 
     |████████████████████████████████| 1.2 MB 40.9 MB/s 
     |████████████████████████████████| 130 kB 63.9 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 99 kB 10.6 MB/s 
     |████████████████████████████████| 428 kB 52.1 MB/s 
     |████████████████████████████████| 344 kB 50.2 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s

The following functions are responsible to access and display [Cloud-Optimized GeoTIFF (COG)](https://www.cogeo.org/) publicly accessible through a remote URL:

In [ ]:
def read_overview(cog_url, level=3):
  import rasterio
  with rasterio.open(cog_url) as src:
    oviews = src.overviews(1)
    oview = oviews[level]

    return src.read(1, out_shape=(1, src.height // oview, src.width // oview))

def display_cog(urls, palettes='magma', basemap = 'HYBRID'):
  from pathlib import Path
  import numpy as np
  import leafmap

  if not isinstance(urls, list):
    urls = [ urls ]
  if not isinstance(palettes, list):
    palettes = [ palettes ]

  Map = leafmap.Map()
  if basemap is not None:
    Map.add_basemap(basemap)
    
  for i in range(0, len(urls)):
    url = urls[i]
    name = Path(url).name
    data = read_overview(url,level=-2)
    vmin, vmax = np.percentile(data.shape, [5,95])
    
    if i >= len(palettes):
      palette = palettes[0]
    else:
      palette = palettes[i]


    Map.add_cog_layer(url, name=name, palette=palette, vmin=vmin, vmax=vmax)
  
  return Map


## Cloud-free Landsat ARD (2000–2020)

The first dataset that you will access is the [**quarterly green band of GLAD Landsat ARD (2000–2020)**](https://stac.opendatascience.eu/lcv_red_landsat.glad.ard/collection.json). Use this link to choose a specific period and band (e.g. ``green_p50``) in ODSE STAC Browser, copying the download link to the variable ``url``.

The ``display_cog`` function accepts a [palettes](https://matplotlib.org/3.5.0/tutorials/colors/colormaps.html) and [basemap](https://leafmap.org/notebooks/02_using_basemaps/) arguments. 

In [ ]:
url = 'https://s3.eu-central-1.wasabisys.com/eumap/lcv/lcv_green_landsat.glad.ard_p50_30m_0..0cm_2019.06.25..2019.09.12_eumap_epsg3035_v1.1.tif'
display_cog(url)

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [ ]:
display_cog(url, palettes='cividis')

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

It's also possible add multiple COG in the same map:

In [ ]:
urls = [
        'https://s3.eu-central-1.wasabisys.com/eumap/lcv/lcv_green_landsat.glad.ard_p50_30m_0..0cm_2000.06.25..2000.09.12_eumap_epsg3035_v1.1.tif',
        'https://s3.eu-central-1.wasabisys.com/eumap/lcv/lcv_green_landsat.glad.ard_p50_30m_0..0cm_2019.06.25..2019.09.12_eumap_epsg3035_v1.1.tif'
]
display_cog(urls)

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Cloud-free Sentinel-2 L2A (2018-2021)

The [Open Data Science Europe (ODSE)](https://opendatascience.eu/) also provides [quarterly cloud-free Sentinel-2 L2A](https://stac.opendatascience.eu/lcv_green_sentinel.s2l2a/collection.json) mosaics. Use this link to select the same period and band of the Landsat mosaic, copying the download link to the variable url.

In [ ]:
urls = [
        'https://s3.eu-central-1.wasabisys.com/eumap/lcv/lcv_green_landsat.glad.ard_p50_30m_0..0cm_2000.06.25..2000.09.12_eumap_epsg3035_v1.1.tif',
        'https://s3.eu-central-1.wasabisys.com/eumap/lcv/lcv_green_sentinel.s2l2a_p50_30m_0..0cm_2019.06.25..2019.09.12_eumap_epsg3035_v1.0.tif'
]
display_cog(urls)

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Annual broad-leaved forest at 30 m (2000–2020)

The land cover and land use product mapped 43 classes including dominant classes, probabilities and uncertainties ([Witjes et al., 2022](https://www.researchsquare.com/article/rs-561383/v4)). To demonstrate how you can access it, let's use the [broad-leaved forest class](https://stac.opendatascience.eu/lcv_landcover.311_lucas.corine.eml/collection.json) available in [ODSE-STAC](https://stac.opendatascience.eu/).

In [ ]:
url = 'https://s3.eu-central-1.wasabisys.com/eumap/lcv/lcv_landcover.311_lucas.corine.eml_p_30m_0..0cm_2020_eumap_epsg3035_v0.2.tif'
display_cog(url, palettes='greens')

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [ ]:
urls = [
        'https://s3.eu-central-1.wasabisys.com/eumap/lcv/lcv_landcover.311_lucas.corine.eml_p_30m_0..0cm_2020_eumap_epsg3035_v0.2.tif',
        'https://s3.eu-central-1.wasabisys.com/eumap/lcv/lcv_landcover.311_lucas.corine.eml_md_30m_0..0cm_2020_eumap_epsg3035_v0.2.tif'
]
display_cog(urls, palettes=['greens', 'viridis'])

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Soil log organic carbon content (2000–2020)

The predictive soil mapping performed by ODSE produced the first continental **3D+t product** for multiple soil variables. To demonstrate how you can access it, let's use the [Soil log organic carbon content ](https://stac.opendatascience.eu/sol_log.oc_eumap/collection.json) available in [ODSE-STAC](https://stac.opendatascience.eu/).

In [ ]:
urls = [
        'https://s3.eu-central-1.wasabisys.com/eumap/sol/sol_log.oc_lucas.iso.10694_m_30m_s0..0cm_2020_eumap_epsg3035_v0.2.tif',
        'https://s3.eu-central-1.wasabisys.com/eumap/sol/sol_log.oc_lucas.iso.10694_md_30m_s0..0cm_2020_eumap_epsg3035_v0.2.tif',
        'https://s3.eu-central-1.wasabisys.com/eumap/sol/sol_log.oc_lucas.iso.10694_m_30m_s30..30cm_2020_eumap_epsg3035_v0.2.tif',
        'https://s3.eu-central-1.wasabisys.com/eumap/sol/sol_log.oc_lucas.iso.10694_md_30m_s30..30cm_2020_eumap_epsg3035_v0.2.tif',
]
display_cog(urls, palettes=['terrain', 'viridis', 'terrain', 'viridis'])

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…